In [3]:
import os
import pybaseball
import pandas as pd
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 10)
import numpy as np
import statsapi
from datetime import datetime, timedelta
from dateutil import tz
import time
import re
import warnings
warnings.filterwarnings('ignore')
from api_keys import visual_crossing
import requests

import pybaseball
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 10)
from dateutil import tz
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, plot_confusion_matrix, precision_score, plot_roc_curve, make_scorer
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold

import math

import xgboost as xgb

import category_encoders as ce

from dateutil import tz

from time import sleep

In [3]:
api_counter = 0

In [4]:
def convert_UTC_to_local(row):
    """Takes in a row containing a baseball game's location and UTC start time"""
    
#     Reads in latitude, longitude, and altitude for known venues
    venue_coords = pd.read_csv('Parks.csv')
    
    venue_name = row['venue_name']
    game_datetime = row['game_datetime']
    
#     get timezone object for UTC
    from_zone = tz.gettz('UTC')

    try:
        city = venue_coords[venue_coords['NAME'] == row['venue_name']]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == row['venue_name']]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == row['venue_name']]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == row['venue_name']]['Altitude'].iloc[0]
    except NameError:
#         Need to add new venue to Parks.csv
        raise IndexError(venue, index)
    if city == 'Tokyo':
        to_zone = tz.gettz('Asia/Tokyo')
    elif city == 'London':
        to_zone = tz.gettz('Europe/London')
    elif city in ['San Francisco',
                        'Oakland',
                        'Phoenix',
                        'Seattle',
                        'Los Angeles',
                        'San Diego',
                        'Anaheim']:
        to_zone = tz.gettz('America/Los_Angeles')
    elif city == 'Denver':
        to_zone = tz.gettz('America/Denver')
    elif city in ['Minneapolis',
                        'Milwaukee',
                        'Chicago',
                        'St. Louis',
                        'Arlington',
                        'Kansas City',
                        'Houston',
                        'Monterrey',
                        'Omaha',
                        'Dyersville']:
        to_zone = tz.gettz('America/Chicago')
    elif city in ['Buffalo',
                        'Detroit',
                        'Cincinnati',
                        'Pittsburgh',
                        'Tampa Bay',
                        'Philadelphia',
                        'Atlanta',
                        'New York',
                        'Washington',
                        'Cleveland',
                        'Miami',
                        'Boston',
                        'Baltimore',
                        'Toronto',
                        'Williamsport',
                        'Dunedin',
                        'St. Petersburg']:
        to_zone = tz.gettz('America/New_York')
    else:
        raise NameError(venue_name, city)
    utc = datetime.strptime(game_datetime, '%Y-%m-%dT%H:%M:%SZ')
#     Convert start time from 
    utc = utc.replace(tzinfo=from_zone)
    local = utc.astimezone(to_zone)

    return(local)

In [5]:
# establish definitions for the events column for what ends a plate apearence ending event and a hit
# the statcast includes data for every pitch which needs to be pared down to determine whether a player got a hit
pa_ending_events = np.array(['field_out',
                        'strikeout',
                        'single',
                        'walk',
                        'double',
                        'home_run',
                        'force_out',
                        'grounded_into_double_play',
                        'hit_by_pitch',
                        'field_error',
                        'sac_fly',
                        'triple',
                        'sac_bunt',
                        'fielders_choice',
                        'double_play',
                        'fielders_choice_out',
                        'strikeout_double_play',
                        'catcher_interf',
                        'sac_fly_double_play',
                        'triple_play',
                        'sac_bunt_double_play'])


hit_events = np.array(['single',
            'double',
            'home_run',
            'triple'])

american_league_teams = np.array(['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles'])


In [6]:
def time_date_weather_data(game_pk_df):
    venue_coords = pd.read_csv('Parks.csv')
    data_collection = {}

    try:
        game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join([str(venue_coords[venue_coords['NAME'] == x]['Latitude'].iloc[0]), str(venue_coords[venue_coords['NAME'] == x]['Longitude'].iloc[0])]))
    except NameError:
        raise IndexError(venue, index)

    game_pk_df = game_pk_df.sort_values(['game_datetime'])

    

    game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row, venue_coords), axis=1)
    game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))
    

    from api_keys import visual_crossing
    import requests

    weather_df = pd.DataFrame()
    for index, row in game_pk_df.iterrows():

#         check if key is past or future
        if datetime.strptime(row['game_datetime'], "%Y-%m-%dT%H:%M:%SZ") > datetime.utcnow():
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'

        else:
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={row["local_datetime"]}&endDateTime={row["local_datetime"]}&unitGroup=us&contentType=json&location={row["coordinates"]}&key={visual_crossing}'
        global api_counter
        if api_counter >= 1000:
            print('Prepreprocessing will continue at ' + (datetime.now()+timedelta(seconds = 86400)).strftime('%H:%M:%S'))
            sleep(86400)
            api_counter = 0
        try:
            api_counter += 1
            response = requests.get(URL)
            data = response.json()
            if data['locations'][row['coordinates']]['values'][0]['temp'] == None:
                URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'
                api_counter += 1
                response = requests.get(URL)
                data = response.json()
                
            if len(data['locations'][row['coordinates']]['values']) > 1:
                for forecast in data['locations'][row['coordinates']]['values']:
                    
                    if forecast['datetimeStr'][:13] == row['local_datetime'][:13]:
                        values = forecast
                        values['datetimeStr'] = row['local_datetime']
                
                
            else:
                values = data['locations'][row['coordinates']]['values'][0]
                values['datetimeStr'] = row['local_datetime']
            data_collection[URL] = data
            weather_df = weather_df.append({**{'coordinates': row['coordinates']}, **values}, ignore_index=True)
    
        except:
            raise IndexError(index, row, response, response.json())
    weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})


    games_and_weather = pd.merge(
        game_pk_df,
        weather_df,
        how="left",
        on=None,
        left_on=['local_datetime', 'coordinates'],
        right_on=['local_datetime', 'coordinates'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gpk", "_wdf"),
        copy=True,
        indicator=False,
        validate=None,
    )
    
    return(games_and_weather)

In [4]:
def statcast_importer(year = datetime.now().year):
#     Check if importing current year data
#     if year == datetime.now().year:
#         end = datetime.now()-timedelta(weeks=1) 
#     else:
#         end = datetime(year,12, 31)
#         start = datetime(year, 1, 1)
        
# #     Check if a file has already been started for this data
#     if str(year)+'_statcast_pbp.csv' in os.listdir():
#         df = pd.read_csv(str(year)+'_statcast_pbp.csv')
#         y, m, d = [int(x) for x in df['game_date'].max().split('-')]
        
# #         if the recorded data is within a week of current, make no update

#         if datetime(y, m, d) >= datetime.now()-timedelta(weeks=1):
#             pass
#         else:
# #         otherwise append new data to the file
#             start = datetime(y, m, d) + timedelta(days=1)
#             update = pybaseball.statcast(str(start.year) + '-' + '{:0>2}'.format(str(start.month)) + '-' + '{:0>2}'.format(str(start.day)))
#             update.to_csv('untouched_' + str(y) + '_statcast_pbp.csv', mode='a', header=False)
#             df = pd.read_csv('untouched_' + str(y) + '_statcast_pbp.csv')
#     else:
#         df = pybaseball.statcast(str(year) + '-01-01', str(year) + '-12-31')
#         df.to_csv('untouched_' + str(year) + '_statcast_pbp.csv')

#   fill in missing data with feature median in order to be able to optimize df by specifying types

    start = datetime(year, 1, 1)
    if year == datetime.now().year:
        end = datetime.now()-timedelta(weeks=1)
    else:
        end = datetime(year,12, 31)
    df = pybaseball.statcast(str(start.year) + '-' + '{:0>2}'.format(str(start.month)) + '-' + '{:0>2}'.format(str(start.day)), str(end.year) + '-' + '{:0>2}'.format(str(end.month)) + '-' + '{:0>2}'.format(str(end.day)))
    df.to_csv('untouched_' + str(year) + '_statcast_pbp.csv')
        
    num_columns = ['release_speed', 'batter', 'pitcher', 'zone', 'hit_location', 'balls', 'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'fielder_2', 'hit_distance_sc', 'release_spin_rate', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'woba_value', 'woba_denom', 'babip_value', 'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score']

    num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median'))
    ])
    preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_columns)])
    clf = Pipeline(steps=[('preprocessor', preprocessor)])

    t = clf.fit_transform(df)

    df[num_columns] = t

#   drop spring training, playoff games

    df = df[df['game_type'] == 'R']
    print('Non-regular season games dropped')


    df.to_csv(str(year)+'_statcast_pbp.csv')
    return(df)
        
        

In [37]:
def derived_cumulative_stats_v2(player, stats_type, history, start_date=None, end_date=datetime.strptime('2021-06-30', '%Y-%m-%d')):
    print(end_date)
    print(player)

#    statistics meant to describe last 2 years

    if start_date == None:
        start_date = end_date-timedelta(weeks=102)
        
#    if player's name is passed in, rather than their mlbam ID, lookup ID

    if type(player) == str:
        if player[-3:] == 'Jr.':
            player = player[:-4]
        player = player.split(' ')
        player.reverse()
        try:
            player_info = pybaseball.playerid_lookup(player[0], player[-1])
            if len(player_info) >1:
                player_info = player_info.iloc[0]
                player = player_info['key_mlbam']
            elif player_info.empty:
                player_info = pybaseball.playerid_lookup(player[0], player[-1], fuzzy=True).iloc[0]
                player = player_info['key_mlbam']
            else:
                try:
                    player = player_info.iloc[0]['key_mlbam']
                except:
                    raise NameError(player_info, player)
        except:
            raise NameError(player)
            
    history = history[(history['game_date'] >= start_date) & (history['game_date'] <= end_date)]

    history = history[history['events'].isin([1, 0])]

    
#     if player is a pitcher, send down pitcher path
    if stats_type == 'pitcher':
        try:
            history = history[history['pitcher'] == player]
        except:
            raise NameError(player)
        print('player filtered')
    #   PITCH METRICS
        pitches = {}
        while True:
            try:
                pitches['pitch_hand'] = statsapi.player_stat_data(player)['pitch_hand']
                break
            except:
                print('pitch_hand_error')
                sleep(10)
        
    #     filter history down to plate appearence ending plays
        
        games = history['game_pk'].unique()
        plate_appearences_list = []
        num_hits_list = []
        inning_list = []
        for game in games:
            game = history[history['game_pk'] == game]
            inning = game['inning'].max()
            inning_list.append(inning)
            plate_appearences = len(game)
            plate_appearences_list.append(plate_appearences)
#             for each inning, collect number of hit events to determine average inning
            for i in range(1, inning+1):
                num_hits = len(game[(game['inning'] == i) & (game['events'] == 1)])
                num_hits_list.append(num_hits)
        pitches['games_played_last_2_years_pitcher'] = len(games)
        pitches['avg_innings_per_appearance'] = np.array(inning_list).mean()
        pitches['avg_PAs_per_appearance_pitcher'] = np.array(plate_appearences_list).mean()
        pitches['avg_hits_per_inning'] = np.array(num_hits_list).mean()

#         L/R splits
        left_hitter = history[history['batter_righty'] == 0]
        right_hitter = history[history['batter_righty'] == 1]
        l_pas = len(left_hitter)
        r_pas = len(right_hitter)
        l_hits = len(left_hitter[(history['events'] == 1)])
        r_hits = len(right_hitter[(history['events'] == 1)])

        if l_pas > 0 and r_pas > 0:
            pitches['H/PA_pitcher'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = (l_hits)/(l_pas)
            pitches['against_righties_H/PA'] = (r_hits)/(r_pas)
        elif l_pas > 0:
            pitches['H/PA_pitcher'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = (l_hits)/(l_pas)
            pitches['against_righties_H/PA'] = 0
        elif r_pas > 0:
            pitches['H/PA_pitcher'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = 0
            pitches['against_righties_H/PA'] = (r_hits)/(r_pas)
        else:
            pitches['H/PA_pitcher'] = 0
            pitches['against_lefties_H/PA'] = 0
            pitches['against_righties_H/PA'] = 0
        return(pitches)

    
    elif stats_type == 'batter':
        history = history[history['batter'] == player]


#     H/PA per pitch type
#     K/PA per pitch type
#     H/PA vs righties, lefties
#     PA/G
#     avg_launch_angle
#     avg_launch_speed
#     xBA based on estimated_ba_using_speedangle



        games = history['game_pk'].unique()
        pas_list = []

        batter = {}

#         find average number of plate appearences per game
        for game in games:
            pas = len(history[history['game_pk'] == game])
            pas_list.append(pas)
        batter['games_played_last_2_years_batter'] = len(games)
        batter['PA/G_batter'] = np.array(pas_list).mean()

#         righty/lefty split
        pitcher_right = history[history['pitcher_righty'] == 1]
        pas_r = len(pitcher_right)
        hits_r = len(pitcher_right[pitcher_right['events'] == 1])

        if pas_r > 0:
            batter['H/PA_against_R'] = hits_r/pas_r
        else:
            batter['H/PA_against_R'] = 0

        pitcher_left = history[history['pitcher_righty'] == 0]
        pas_l = len(pitcher_left)
        hits_l = len(pitcher_left[pitcher_left['events'] == 1])

        if pas_l > 0:
            batter['H/PA_against_L'] = hits_l/pas_l
        else:
            batter['H/PA_against_L'] = 0

        if pas_r > 0 or pas_l > 0:
            batter['H/PA_batter'] = (hits_r+hits_l)/(pas_r+pas_l)
        else:
            batter['H/PA_batter'] = 0
        batter['avg_launch_angle'] = history['launch_angle'].mean()
        batter['avg_launch_speed'] = history['launch_speed'].mean()
        batter['xBA'] = history['estimated_ba_using_speedangle'].mean()

        return(batter)

In [56]:
def prepreprocess_historic_data(df):
    df_filtered = df[df['events'].isin([1,0])]

    venue_coords = pd.read_csv('Parks.csv')

    game_pk_df = pd.DataFrame(columns = statsapi.schedule(game_id=566083)[0].keys())

    venue_dict = {}

#             grabs metadata for respective games such as projected starting pitchers and venue names
    for game in df_filtered['game_pk'].unique():
        print(game)
        game_data = statsapi.schedule(game_id = int(game))[-1]
        game_pk_df = game_pk_df.append(game_data, ignore_index=True)
        print(game)

#             grabs coordinate date for venues from Parks.csv which needs to be manually updated every time
#             there's a new venue or name change
    try:
        for venue in game_pk_df['venue_name'].unique():
            print(venue)
            city = venue_coords[venue_coords['NAME'] == venue]['CITY'].iloc[0]
            lat = venue_coords[venue_coords['NAME'] == venue]['Latitude'].iloc[0]
            lon = venue_coords[venue_coords['NAME'] == venue]['Longitude'].iloc[0]
            alt = venue_coords[venue_coords['NAME'] == venue]['Altitude'].iloc[0]
            venue_dict[venue] = np.array((city, lat, lon, alt))
    except IndexError:
        raise IndexError(venue, index)


#             converts game start times from UTC to local times, based on hand written mapping 
#             in convert_UTC_to_local for the purposes of collating start times to cut down on 
#             Visual Crossing api calls as they cost money after the first 1000 per day
#             

    game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)
    game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))


    datetime_coordinate_matching = {}
    for index, row in game_pk_df.iterrows():
        city, lat, lon, alt = venue_dict[row['venue_name']]
        dc_datetime = row['local_datetime']
        if dc_datetime in datetime_coordinate_matching.keys():
            datetime_coordinate_matching[dc_datetime].append(','.join([str(lat), str(lon)]))
        else:
            datetime_coordinate_matching[dc_datetime] = [','.join([str(lat), str(lon)])]

    game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join(venue_dict[x][1:3]))
    game_pk_df['alt'] = game_pk_df['venue_name'].apply(lambda x: venue_dict[x][3])

#             calls Visual Crossing for historical weather data for each game

    from api_keys import visual_crossing
    import requests

    weather_df = pd.DataFrame()
    for key, value in datetime_coordinate_matching.items():
        print(key)
        url_locations = '|'.join(value)
        URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={key}&endDateTime={key}&unitGroup=us&contentType=json&location={url_locations}&key={visual_crossing}'
        global api_counter
        if api_counter == 10000000:
            print('Prepreprocessing will continue at ' + (datetime.now()+timedelta(seconds = 86400)).strftime('%H:%M:%S'))
            sleep(86400)
            api_counter = 0
        try:
            api_counter += 1
            response = requests.get(URL)
            data = response.json()
            locations = list(data['locations'].keys())
            for each in locations:
                values = data['locations'][each]['values'][0]

#                         creates a df containing coordinate data and returned weather data to later be added with
#                         other collected data

                weather_df = weather_df.append({**{'coordinates': each}, **values}, ignore_index=True)
        except:
            raise IndexError(key, value, response)
#             cuts off the trailing stamp in datetimeStr signifying the time zone (e.g.-05:00)
    weather_df['datetimeStr'] = weather_df['datetimeStr'].apply(lambda x: x[:-6])
    weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})

    games_and_weather = pd.merge(
        game_pk_df,
        weather_df,
        how="left",
        on=None,
        left_on=['local_datetime', 'coordinates'],
        right_on=['local_datetime', 'coordinates'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gpk", "_acw"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_detailed = pd.merge(
        games_and_weather,
        df_filtered,
        how="right",
        on=None,
        left_on='game_id',
        right_on='game_pk',
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gaw", "_dff"),
        copy=True,
        indicator=False,
        validate=None,
    )


#             additional useful data added to df
    df_detailed['covid_doubleheader'] = df_detailed.apply(lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] =='Y' else 0, axis=1)
    df_detailed['designated_hitter'] = df_detailed.apply(lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis = 1)


#         read in entire relevent history (2 years prior to game in data) in order to assemble 
#         cumulative stats for players
    history = pd.concat([
                set_types(pd.read_csv('2017_statcast_pbp.csv'), type_dict),
               set_types(pd.read_csv('2018_statcast_pbp.csv'), type_dict),
               set_types(pd.read_csv('2019_statcast_pbp.csv'), type_dict),
               set_types(pd.read_csv('2020_statcast_pbp.csv'), type_dict),
               set_types(pd.read_csv('2021_statcast_pbp.csv'), type_dict)
                ], ignore_index=True)

#         pitcher df combines away and home pitchers into one column
#         each row is a projected starting pitcher and the game they're
#         starting as well as the date
    pitcher_df = pd.concat([df_detailed[['game_id', 'game_date_gaw', 'away_probable_pitcher']].rename(columns={'away_probable_pitcher': 'probable_pitcher'}), df_detailed[['game_id', 'game_date_gaw', 'home_probable_pitcher']].rename(columns={'home_probable_pitcher': 'probable_pitcher'})])
    pitcher_df = pitcher_df.drop_duplicates()

#         if probable pitcher is unknown, drop row
    pitcher_df['probable_pitcher'] = pitcher_df['probable_pitcher'].replace('', np.nan)
    pitcher_df = pitcher_df.dropna(subset=['probable_pitcher'])
    pitcher_df = pitcher_df.reset_index(drop=True)

#         runs pitcher_df through derived_cumulative_stats function to get their recent history
    pitcher_stats = pitcher_df.apply(lambda row: pd.Series(derived_cumulative_stats_v2(row['probable_pitcher'], 'pitcher', history, end_date=datetime.strptime(row['game_date_gaw'], '%Y-%m-%d')-timedelta(days=1))), axis = 1)

    pitcher_df = pd.merge(
        pitcher_stats,
        pitcher_df,
        how="left",
        on=None,
        left_on=None,
        right_on=None,
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("_s", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )

#         re-matches home batters with away pitchers and vice versa

    home_batters = df_detailed[df_detailed['home'] == 1]
    away_batters = df_detailed[df_detailed['home'] == 0]

    df_wpitching_p1 = pd.merge(
        pitcher_df,
        home_batters,
        how="right",
        on=None,
        left_on=['game_id', 'probable_pitcher'],
        right_on=['game_id', 'away_probable_pitcher'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_p", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_wpitching_p2 = pd.merge(
        pitcher_df,
        away_batters,
        how="right",
        on=None,
        left_on=['game_id', 'probable_pitcher'],
        right_on=['game_id', 'home_probable_pitcher'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_p", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )
    df_wpitching = pd.concat([df_wpitching_p1, df_wpitching_p2], ignore_index=True)
    df_wpitching = df_wpitching.sort_values(['game_date_gaw_df', 'game_pk','home_name', 'inning', 'home', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

    df_wpitching = df_wpitching.rename(columns={'events': 'got_a_hit'})

#         drops repeat batters in the same game, while preserving the 1 value for got_a_hit, if present
    df_wpitching = df_wpitching.sort_values('got_a_hit').drop_duplicates(subset=['game_id', 'batter'], keep='last').sort_values(['game_date_gaw_df', 'game_pk','home_name', 'inning', 'home', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)
        
#     combines batting stats with pitching/statcast/statsapi/weather data
    print(df_wpitching.columns.to_list())
    batting_stats = df_wpitching.apply(lambda row: pd.Series(derived_cumulative_stats_v2(row['batter'], 'batter', history, end_date=datetime.strptime(row['game_date_gaw'], '%Y-%m-%d')-timedelta(days=1))), axis = 1)
    df_final = pd.merge(
        batting_stats,
        df_wpitching,
        how="right",
        on=None,
        left_on=None,
        right_on=None,
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("_bs", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )

#     clarifies meaning of certain columns
    df_final = df_final.rename(columns={'games_played_last_2_years_df': 'games_played_last_2_years_pitcher', 'games_played_last_2_years_bs': 'games_played_last_2_years_batter'})

#     transforms date and datetime columns into easily comparable numbers
    df_final['local_datetime'] = pd.to_datetime(df_final['local_datetime'])
    df_final['local_date'] = df_final['local_datetime'].apply(lambda x: x.toordinal())
    df_final['local_datetime'] = df_final['local_datetime'].apply(lambda x: x.timestamp())

#     drop columns that are functionally duplicates, contain information that wouldn't be known prior to game,
#     or are irrelevent
    df_final =df_final.drop(columns= ['player_name',
     'game_type_dff', 'pitch_type',
     'release_speed',
     'release_pos_x',
     'release_pos_z',
     'pitcher',
     'events',
     'description',
     'spin_dir',
    'spin_rate_deprecated',
     'break_angle_deprecated',
     'break_length_deprecated',
     'zone',
     'des',
      'p_throws',
    'type',
     'hit_location',
     'bb_type',
     'balls',
     'strikes',
     'pfx_x',
     'pfx_z',
     'plate_x',
     'plate_z',
     'on_3b',
     'on_2b',
     'on_1b',
     'outs_when_up',
     'inning',
     'hc_x',
     'hc_y',
     'tfs_deprecated',
     'tfs_zulu_deprecated',
      'sv_id',
     'vx0',
     'vy0',
     'vz0',
     'ax',
     'ay',
     'az',
     'sz_top',
     'sz_bot',
     'hit_distance_sc',
     'launch_speed',
     'launch_angle',
     'effective_speed',
     'release_spin_rate',
     'release_extension',
     'pitcher.1',
     'fielder_2.1',
     'fielder_3',
     'fielder_4',
     'fielder_5',
     'fielder_6',
     'fielder_7',
     'fielder_8',
     'fielder_9',
     'release_pos_y',
     'estimated_ba_using_speedangle',
     'estimated_woba_using_speedangle',
     'woba_value',
     'woba_denom',
     'babip_value',
     'iso_value',
     'launch_speed_angle',
     'at_bat_number',
     'pitch_number',
     'pitch_name',
     'home_score_dff',
     'away_score_dff',
     'bat_score',
     'fld_score',
     'post_away_score',
     'post_home_score',
     'post_bat_score',
     'post_fld_score',
     'if_fielding_alignment',
     'of_fielding_alignment',
     'spin_axis',
     'delta_home_win_exp',
     'delta_run_exp',
     'game_pk',
     'game_id','game_date_gaw_p',
     'game_datetime',
     'game_date_gaw_df',
     'game_type_gaw',
     'datetime',
    #  'datetimeStr',
     'game_date_dff','winning_team',
     'losing_team',
     'winning_pitcher',
     'losing_pitcher',
     'save_pitcher',
     'summary', 'home_probable_pitcher',
     'away_probable_pitcher',
     'home_pitcher_note',
     'away_pitcher_note',
     'away_score_gaw',
     'home_score_gaw',
     'current_inning',
     'inning_state',
     'venue_id',
     'status',
     'home_team', 'away_team', 'home_id', 'away_id'])
#     if all values in a column are identical, drop the column
    for col in df_final.columns.to_list():
        if len(df_final[col].unique()) == 1:
            df_final = df_final.drop([col], axis=1)

In [5]:
for year in [2021]:
    statcast_importer(year)

This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 172/172 [00:32<00:00,  5.28it/s]


Non-regular season games dropped


In [58]:
def lookup_mlb():
    """Get list of MLB team IDs for the purposes of pulling active rosters"""
    params = {
        "activeStatus": 'Y',
        "season": 2021,
        "sportIds": 1,
        "fields": "teams,id,name,league",
    }
    
    r= statsapi.get('teams', params)
    
    teams = {}
    for team in r['teams']:
        if team['league']['id'] in [103, 104]:
            teams[team['name']] = team['id']
    
    return (teams)

def roster(teamId, rosterType=None, season=datetime.now().year, date=None):
    """Get the roster for a given team."""
    if not rosterType:
        rosterType = "active"

    params = {"rosterType": rosterType, "season": season, "teamId": teamId}
    if date:
        params.update({"date": date})

    r = statsapi.get("team_roster", params)

    players = []
    for x in r["roster"]:
        players.append(
            x["person"]["id"]
        )

    
    return (players)

def lookup_team(lookup_value, activeStatus="Y", season=datetime.now().year, sportIds=1):
    """Get a info about a team or teams based on the team name, city, abbreviation, or file code."""
    params = {
        "activeStatus": activeStatus,
        "season": season,
        "sportIds": sportIds,
        "fields": "teams,id,name,teamCode,fileCode,teamName,locationName,shortName,league",
    }
    r = statsapi.get("teams", params)

    teams = []
    for team in r["teams"]:
        for v in team.values():
            if str(lookup_value).lower() in str(v).lower():
                teams.append(team)
                break

    return (teams)

In [59]:
num_columns = ['release_speed', 'batter', 'pitcher', 'zone', 'hit_location', 'balls', 'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'fielder_2', 'hit_distance_sc', 'release_spin_rate', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'woba_value', 'woba_denom', 'babip_value', 'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score']

column_types = [np.int8,
np.int32,
np.int32,
np.int8,
np.int8,
np.int8,
np.int8,
np.int16,
np.int32,
np.int32,
np.int32,
np.int32,
np.int32,
np.int32,
np.int16,
np.int16,
np.int32,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int8,
np.int8,
np.int8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8]

type_dict = dict(zip(num_columns, column_types))

In [60]:

    
#     df['release_speed'] = df['release_speed'].astype(np.int8)
#     #     df['release_pos_x'] = df['release_pos_x'].astype(np.int8)
#     #     df['release_pos_z'] = df['release_pos_z'].astype(np.int8)
#     df['batter'] = df['batter'].astype(np.int32)
#     df['pitcher'] = df['pitcher'].astype(np.int32)
#     df['zone'] = df['zone'].astype(np.int8)
#     df['hit_location'] = df['hit_location'].astype(np.int8)
#     df['balls'] = df['balls'].astype(np.uint8)
#     df['strikes'] = df['strikes'].astype(np.uint8)
#     df['game_year'] = df['game_year'].astype(np.int16)
#     df['on_3b'] = df['on_3b'].astype(np.int32)
#     df['on_2b'] = df['on_2b'].astype(np.int32)
#     df['on_1b'] = df['on_1b'].astype(np.int32)
#     df['outs_when_up'] = df['outs_when_up'].astype(np.uint8)
#     df['inning'] = df['inning'].astype(np.uint8)
#     df['fielder_2'] = df['fielder_2'].astype(np.int32)
#     df['hit_distance_sc'] = df['hit_distance_sc'].astype(np.int16)
#     df['release_spin_rate'] = df['release_spin_rate'].astype(np.int16)
#     df['game_pk'] a= df['game_pk'].astype(np.int32)
#     df['pitcher.1'] = df['pitcher.1'].astype(np.int32)
#     df['fielder_2.1'] = df['fielder_2.1'].astype(np.int32)
#     df['fielder_3'] = df['fielder_3'].astype(np.int32)
#     df['fielder_4'] = df['fielder_4'].astype(np.int32)
#     df['fielder_5'] = df['fielder_5'].astype(np.int32)
#     df['fielder_6'] = df['fielder_6'].astype(np.int32)
#     df['fielder_7'] = df['fielder_7'].astype(np.int32)
#     df['fielder_8'] = df['fielder_8'].astype(np.int32)
#     df['fielder_9'] = df['fielder_9'].astype(np.int32)
#     df['woba_value'] = df['woba_value'].astype(np.int8)
#     df['woba_denom'] = df['woba_denom'].astype(np.int8)
#     df['babip_value'] = df['babip_value'].astype(np.int8)
#     df['at_bat_number'] = df['at_bat_number'].astype(np.uint8)
#     df['pitch_number'] = df['pitch_number'].astype(np.uint8)
#     df['home_score'] = df['home_score'].astype(np.uint8)
#     df['away_score'] = df['away_score'].astype(np.uint8)
#     df['bat_score'] = df['bat_score'].astype(np.uint8)
#     df['fld_score'] = df['fld_score'].astype(np.uint8)
#     df['post_away_score'] = df['post_away_score'].astype(np.uint8)
#     df['post_home_score'] = df['post_home_score'].astype(np.uint8)
#     df['post_bat_score'] = df['post_bat_score'].astype(np.uint8)
#     df['post_fld_score'] = df['post_fld_score'].astype(np.uint8)

def set_types(df, type_dict):
    '''Explicitly names datatypes in order to speed up preprocessing of a large dataset'''
    
    df['game_date'] = pd.to_datetime(df['game_date'], format='%Y-%m-%d')
    print('datetime done')
    df['events'] = df['events'].apply(lambda x: 1 if x in hit_events else 0).astype(np.int8)
    print('events done')
    df['batter_righty'] = df['stand'].apply(lambda x: 1 if x == 'R' else 0).astype(np.int8)
    print('stand done')
    df['pitcher_righty'] = df['p_throws'].apply(lambda x: 1 if x == 'R' else 0).astype(np.int8)
    print('pitcher done')
    df['home'] = df['inning_topbot'].apply(lambda x: 1 if x == 'Bot' else 0).astype(np.int8)
    print('inning done')
    
    for key, value in type_dict.items():
        df[key] = df[key].astype(value)
    
    df = df.select_dtypes(exclude=['object'])

    return (df)

In [61]:
df = set_types(pd.read_csv('2021_statcast_pbp.csv'), type_dict)

df = df[df['game_date']<= datetime(2021,6,30)]

start = datetime.now()
sample = prepreprocess_historic_data(pd.concat([
#                 set_types(pd.read_csv('2019_statcast_pbp.csv')),
                set_types(pd.read_csv('2020_statcast_pbp.csv'), type_dict),
               df
                ], ignore_index=True))
print(datetime.now() - start)
start = datetime.now()

df = df[df['game_date']> datetime(2021,6,30)]

holdout = prepreprocess_historic_data(df)
print(datetime.now() - start)

sample.to_csv('sample', index_col=0)
holdout.to_csv('holdout', index_col=0)

datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
633291
633291
632919
632919
632909
632909
632908
632908
632893
632893
632884
632884
632880
632880
632875
632875
632872
632872
632871
632871
632869
632869
632868
632868
632864
632864
632853
632853
632851
632851
632825
632825
633421
633421
632892
632892
632890
632890
632886
632886
632883
632883
632882
632882
632878
632878
632873
632873
632858
632858
632813
632813
632961
632961
632917
632917
632910
632910
632899
632899
632895
632895
632891
632891
632881
632881
632876
632876
632874
632874
632867
632867
632866
632866
632863
632863
632860
632860
632857
632857
632847
632847
632958
632958
632941
632941
632925
632925
632916
632916
632913
632913
632907
632907
632905
632905
632898
632898
632897
632897
632894
632894
632889
632889
632885
632885
632877
632877
632820
632820
632819
632819
632818
632818
632972
632972
632954
632954
632943
632943
632936
632936
632932
632932
632930
6

633470
633468
633468
633464
633464
633462
633462
633452
633452
633451
633451
633448
633448
633433
633433
633420
633420
633415
633415
633404
633404
633402
633402
633529
633529
633481
633481
633480
633480
633469
633469
633463
633463
633460
633460
633457
633457
633456
633456
633438
633438
633418
633418
633405
633405
633506
633506
633491
633491
633485
633485
633484
633484
633479
633479
633474
633474
633467
633467
633461
633461
633445
633445
633437
633437
633435
633435
633432
633432
633427
633427
633426
633426
633419
633419
633587
633587
633508
633508
633497
633497
633496
633496
633495
633495
633488
633488
633478
633478
633471
633471
633455
633455
633454
633454
633449
633449
633446
633446
633444
633444
633443
633443
634620
634620
633867
633867
633601
633601
633490
633490
633483
633483
633475
633475
633466
633466
633465
633465
633458
633458
633450
633450
633544
633544
633527
633527
633516
633516
633515
633515
633504
633504
633499
633499
633494
633494
633493
633493
633489
633489
633486
633486

634055
634053
634053
634046
634046
634044
634044
634037
634037
634034
634034
634033
634033
634031
634031
634030
634030
634029
634029
634026
634026
634133
634133
634115
634115
634109
634109
634107
634107
634102
634102
634092
634092
634075
634075
634073
634073
634067
634067
634065
634065
634058
634058
634052
634052
634051
634051
634050
634050
634043
634043
634101
634101
634099
634099
634091
634091
634082
634082
634077
634077
634076
634076
634070
634070
634063
634063
634057
634057
634188
634188
634134
634134
634130
634130
634124
634124
634122
634122
634100
634100
634098
634098
634094
634094
634090
634090
634087
634087
634086
634086
634083
634083
634071
634071
634069
634069
634068
634068
634161
634161
634148
634148
634143
634143
634139
634139
634127
634127
634111
634111
634106
634106
634105
634105
634104
634104
634097
634097
634096
634096
634095
634095
634085
634085
634084
634084
634080
634080
634656
634656
634206
634206
634166
634166
634162
634162
634146
634146
634145
634145
634138
634138

634551
634651
634651
634637
634637
634617
634617
634613
634613
634611
634611
634610
634610
634607
634607
634597
634597
634593
634593
634572
634572
634560
634560
634557
634557
634654
634654
634646
634646
634644
634644
634643
634643
634639
634639
634633
634633
634629
634629
634623
634623
634604
634604
634596
634596
634580
634580
634577
634577
634575
634575
634573
634573
634653
634653
634631
634631
634627
634627
634605
634605
634602
634602
634578
634578
634576
634576
634655
634655
634652
634652
634645
634645
634642
634642
634640
634640
634638
634638
634636
634636
634634
634634
634630
634630
634625
634625
634622
634622
634618
634618
634615
634615
632887
632887
632865
632865
632862
632862
632861
632861
632859
632859
632856
632856
632855
632855
632854
632854
632852
632852
632850
632850
632849
632849
632848
632848
632844
632844
632839
632839
632831
632831
632846
632846
632845
632845
632843
632843
632841
632841
632840
632840
632838
632838
632835
632835
632833
632833
632823
632823
632816
632816

2021-06-26T15:05:00
2021-06-26T19:15:00
2021-06-26T13:10:00
2021-06-26T15:07:00
2021-06-26T15:10:00
2021-06-26T19:10:00
2021-06-26T18:10:00
2021-06-26T19:05:00
2021-06-24T23:33:00
2021-06-25T19:10:00
2021-06-25T15:10:00
2021-06-25T19:05:00
2021-06-25T16:10:00
2021-06-25T19:07:00
2021-06-25T18:45:00
2021-06-25T19:15:00
2021-06-24T19:10:00
2021-06-24T19:07:00
2021-06-24T13:05:00
2021-06-24T19:15:00
2021-06-23T19:10:00
2021-06-23T13:10:00
2021-06-23T12:40:00
2021-06-23T13:07:00
2021-06-23T12:35:00
2021-06-23T19:05:00
2021-06-23T13:05:00
2021-06-22T19:10:00
2021-06-22T18:38:00
2021-06-22T19:05:00
2021-06-22T18:40:00
2021-06-22T12:10:00
2021-06-20T23:33:00
2021-06-21T17:10:00
2021-06-21T19:05:00
2021-06-21T19:10:00
2021-06-21T18:40:00
2021-06-20T13:05:00
2021-06-20T13:20:00
2021-06-20T14:10:00
2021-06-20T13:10:00
2021-06-20T13:35:00
2021-06-20T19:08:00
2021-06-20T13:07:00
2021-06-19T13:05:00
2021-06-19T19:10:00
2021-06-19T19:07:00
2021-06-19T13:20:00
2021-06-19T16:15:00
2021-06-19T15:05:00


...

player filtered
2021-05-14 00:00:00
Carlos Rodon
player filtered
2021-05-14 00:00:00
Jose Urena
player filtered
2021-05-14 00:00:00
Brett Anderson
player filtered
2021-05-14 00:00:00
Shane McClanahan
player filtered
2021-05-14 00:00:00
Jorge Lopez
player filtered
2021-05-14 00:00:00
Seth Frankoff
player filtered
2021-05-14 00:00:00
Trevor Bauer
player filtered
2021-05-14 00:00:00
Martin Perez
player filtered
2021-05-14 00:00:00
Tyler Anderson
player filtered
2021-05-14 00:00:00
Justus Sheffield
player filtered
2021-05-14 00:00:00
Jose Berrios
player filtered
2021-05-13 00:00:00
Lucas Giolito
player filtered
2021-05-13 00:00:00
Tarik Skubal
player filtered
2021-05-13 00:00:00
Michael Kopech
player filtered
2021-05-13 00:00:00
Riley Smith
player filtered
2021-05-13 00:00:00
Dean Kremer
player filtered
2021-05-13 00:00:00
Clayton Kershaw
player filtered
2021-05-13 00:00:00
Nick Pivetta
player filtered
2021-05-13 00:00:00
Joe Musgrove
player filtered
2021-05-13 00:00:00
Matt Shoemaker
play

player filtered
2021-05-02 00:00:00
Frankie Montas
player filtered
2021-05-02 00:00:00
Adam Wainwright
player filtered
2021-05-02 00:00:00
Kenta Maeda
player filtered
2021-05-02 00:00:00
Daniel Lynch
player filtered
2021-05-02 00:00:00
Vince Velasquez
player filtered
2021-05-02 00:00:00
Miguel Diaz
player filtered
2021-05-02 00:00:00
Erik Swanson
player filtered
2021-05-01 00:00:00
Jose Berrios
player filtered
2021-05-01 00:00:00
Justus Sheffield
player filtered
2021-05-01 00:00:00
Rich Hill
player filtered
2021-05-01 00:00:00
Corey Kluber
player filtered
2021-05-01 00:00:00
Sean Manaea
player filtered
2021-05-01 00:00:00
Lucas Giolito
player filtered
2021-05-01 00:00:00
Joe Musgrove
player filtered
2021-05-01 00:00:00
Merrill Kelly
player filtered
2021-05-01 00:00:00
Tyler Mahle
player filtered
2021-05-01 00:00:00
Alec Bettinger
player filtered
2021-05-01 00:00:00
Mike Foltynewicz
player filtered
2021-05-01 00:00:00
Wil Crowe
player filtered
2021-05-01 00:00:00
Max Scherzer
player fil

player filtered
2021-04-20 00:00:00
Jakob Junis
player filtered
2021-04-20 00:00:00
Tyler Mahle
player filtered
2021-04-20 00:00:00
Frankie Montas
player filtered
2021-04-20 00:00:00
Jose Quintana
player filtered
2021-04-20 00:00:00
Zach Eflin
player filtered
2021-04-20 00:00:00
Austin Gomber
player filtered
2021-04-20 00:00:00
Spencer Turnbull
player filtered
2021-04-20 00:00:00
Max Scherzer
player filtered
2021-04-20 00:00:00
Michael Fulmer
player filtered
2021-04-19 00:00:00
Patrick Corbin
player filtered
2021-04-19 00:00:00
Sean Manaea
player filtered
2021-04-19 00:00:00
Jesus Luzardo
player filtered
2021-04-19 00:00:00
Zack Wheeler
player filtered
2021-04-19 00:00:00
Nick Neidert
player filtered
2021-04-19 00:00:00
Jameson Taillon
player filtered
2021-04-19 00:00:00
Marco Gonzales
player filtered
2021-04-19 00:00:00
Jake Arrieta
player filtered
2021-04-19 00:00:00
Eduardo Rodriguez
player filtered
2021-04-20 00:00:00
Luis Castillo
player filtered
2021-04-19 00:00:00
Brad Keller
pl

2021-04-08 00:00:00
Zach Plesac
player filtered
2021-04-08 00:00:00
Rich Hill
player filtered
2021-04-08 00:00:00
Johnny Cueto
player filtered
2021-04-08 00:00:00
Kohei Arihara
player filtered
2021-04-07 00:00:00
Taijuan Walker
player filtered
2021-04-07 00:00:00
Adam Wainwright
player filtered
2021-04-07 00:00:00
Lance Lynn
player filtered
2021-04-07 00:00:00
Jon Gray
player filtered
2021-04-07 00:00:00
Tyler Anderson
player filtered
2021-04-07 00:00:00
Jose Berrios
player filtered
2021-04-07 00:00:00
Cristian Javier
player filtered
2021-04-07 00:00:00
Ross Stripling
player filtered
2021-04-07 00:00:00
Matt Harvey
player filtered
2021-04-06 00:00:00
Nathan Eovaldi
player filtered
2021-04-06 00:00:00
Aaron Nola
player filtered
2021-04-06 00:00:00
Matthew Boyd
No identically matched names found! Returning the 5 most similar names.
player filtered
2021-04-06 00:00:00
Stephen Strasburg
player filtered
2021-04-06 00:00:00
Justin Dunn
player filtered
2021-04-06 00:00:00
Shane Bieber
player 

KeyError: 'game_date_gaw'

# MODEL WORK

In [ ]:
def evaluate(estimator, X_train, X_val, y_train, y_val, holdout, roc_auc='proba', output = False):

    X_holdout = holdout.drop(['got_a_hit'], axis=1)
    y_holdout = holdout['got_a_hit']
    
    #     grab predictions
    train_preds = estimator.predict(X_train)
    val_preds = estimator.predict(X_val)
    holdout_preds = estimator.predict(X_holdout)
    
#     output needed for roc_auc score
    if roc_auc == 'skip':
        train_out = False
        val_out = False
        holdout_out = False
    elif roc_auc == 'dec': # not all classifiers have a decision function
        train_out = estimator.decision_function(X_train)
        val_out = estimator.decision_function(X_val)
        holdout_out = estimator.decision_function(X_holdout)
    elif roc_auc == 'proba':
        try:
            train_out = estimator.predict_proba(X_train)[:, 1]
            val_out = estimator.predict_proba(X_val)[:, 1]
            holdout_out = estimator.predict_proba(X_holdout)[:, 1]
        except AttributeError:
            train_out = estimator.predict(X_train)
            val_out = estimator.predict(X_val)
            holdout_out = estimator.predict(X_holdout)

    else:
        raise Exception("The value for roc_auc should be 'skip', 'dec', or 'proba'.")
    
    ac = accuracy_score(y_train, train_preds)
    f1 = f1_score(y_train, train_preds)
    ras = roc_auc_score(y_train, train_out)
    pr = precision_score(y_train, train_preds)
    
    if output == True:
        print('Train Scores')
        print('------------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(train_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

    ac = accuracy_score(y_val, val_preds)
    f1 = f1_score(y_val, val_preds)
#     print(type(y_val))
    ras = roc_auc_score(y_val, val_out)
    pr = precision_score(y_val, val_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Val Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(val_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')
    
    ac = accuracy_score(y_holdout, holdout_preds)
    f1 = f1_score(y_holdout, holdout_preds)
#     print(type(holdout['got_a_hit']))
    ras = roc_auc_score(y_holdout, holdout_out)
    pr = precision_score(y_holdout, holdout_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Holdout Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(holdout_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

        print('\nVal Data')
        print('-----------')

        plot_confusion_matrix(estimator, X_val, y_val, values_format=',.5g')
        plt.show()
        
        print('Holdout Data')        
        print('-----------')

        plot_confusion_matrix(estimator, X_holdout, y_holdout, values_format=',.5g')
        plt.show()

In [ ]:
def streak_checker(dataframe, output = False, metric = 'odds', ascending=False, combiner=None, timed=False):
    results = []

    try:
        if 'L/R_split' in metric and combiner == 'add':
            if timed == True:
                start = datetime.now()

                print(combiner, start)
            dataframe_copy = dataframe.copy()
            dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] + row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['stand'] == 'L') else (row['H/PA_against_L'] + row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['stand'] == 'R') else 0), axis=1)
            mms = MinMaxScaler()
            dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
            
            if timed == True:
                print(datetime.now() - start)
            dataframe = dataframe_copy
            metric='temp'
        elif 'L/R_split' in metric and combiner == 'multiply':
            if timed == True:
                start = datetime.now()

                print(combiner, start)
            dataframe_copy = dataframe.copy()
            dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] * row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['stand'] == 'L') else (row['H/PA_against_L'] * row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['stand'] == 'R') else 0), axis=1)
            mms = MinMaxScaler()
            dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
            
            if timed == True:
                print(datetime.now() - start)
            dataframe = dataframe_copy
            metric='temp'
        if type(metric) == list:
            dataframe_copy = dataframe.copy()
            if combiner == 'add':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] + row[metric[1]], axis=1)
                mms = MinMaxScaler()
                dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
                
                if timed == True:
                    print(datetime.now() - start)
            elif combiner == 'multiply':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] * row[metric[1]], axis=1)
                mms = MinMaxScaler()
                dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
                
                if timed == True:
                    print(datetime.now() - start)
            
            else:
                raise KeyError(f'{combiner} combiners have not been implemented yet')
            dataframe = dataframe_copy
            metric = 'temp'
        else:
            pass
    except KeyError:
        raise KeyError("metric must be chosen from dataframe's columns")
    
    date_list = np.sort(dataframe['local_date'].unique())
    
    for day in date_list:
        best_bet = dataframe[dataframe['local_date'] == day].sort_values(metric, ascending=ascending).iloc[0]
        
        odds = best_bet[metric]
        date = datetime.fromordinal(best_bet['local_date']).strftime("%b %d %Y")
        
        player_info = pybaseball.playerid_reverse_lookup([best_bet['batter']])
        if len(player_info) >1:
            player_info = player_info.iloc[0]
            player = player_info['key_mlbam']
        elif player_info.empty:
            player_info = playerid_lookup(player[0], player[1], fuzzy=True).iloc[0]
            player = player_info['key_mlbam']
        else:
            try:
                player = ' '.join(player_info.iloc[0][['name_first', 'name_last']])
            except:
                raise NameError(player_info, player)
        if best_bet['home'] == 0:
            team = best_bet['away_name']
        else:
            team = best_bet['home_name']
    #     team = statsapi.lookup_team(player['currentTeam']['id'])[0]['name']
    #     player_name = player['fullName']
        actual_result = best_bet['got_a_hit']
        results.append(actual_result)
        if output == True:
            if day > date_list[-20]:
                pass
#                 print(date, player, odds, actual_result)
    longest = 0
    current = 0
    for num in results:
        if num == 1:
            current += 1
        else:
            longest = max(longest, current)
            current = 0
    if output == True:
        print(f'Longest streak in set ({len(results)} days): ', longest)
        print(f'Total correct guesses in set ({len(results)} days): ', sum(results), sum(results)/len(results))
#     return (dataframe, best_bet)

In [ ]:
def run_model(clf, X_train, X_val, y_train, y_val, holdout):
    probability = True
    
    clf.fit(X_train.head(100), y_train.head(100))

    evaluate(clf, X_train.head(100), X_val.head(100), y_train.head(100), y_val.head(100), holdout);

    runtime = datetime.now() - start

    if runtime > timedelta(seconds=5):
        print(f'Model would take {runtime.total_seconds()*200/60} minutes to run.')
    else:
        print('Model should take between {:.0f} and {:.0f} minutes to run and finish by {}'.format(runtime.total_seconds()*20/60, runtime.total_seconds()*200/60, (datetime.now()+(timedelta(seconds=(runtime.total_seconds()*200)))).strftime("%H:%M")))
        clf.fit(X_train, y_train)
        evaluate(clf, X_train, X_val, y_train, y_val, holdout, output=True)
        plot_roc_curve(clf, X_val, y_val)
        plt.show()
        
        try:
            val_df_odds = pd.Series(clf.predict_proba(X_val)[:, 1], name='odds')
            holdout_df_odds = pd.Series(clf.predict_proba(holdout.drop(['got_a_hit'], axis=1))[:, 1], name='odds')
        except AttributeError:
            probability = False
            val_df_odds = pd.Series(clf.predict(X_val), name= 'odds')
            holdout_df_odds = pd.Series(clf.predict(holdout.drop(['got_a_hit'], axis=1)), name='odds')

        val_df = X_val.assign(got_a_hit = y_val).reset_index(drop=True)
        val_df = val_df.assign(odds=val_df_odds)
        holdout = holdout.reset_index(drop=True)
        holdout = holdout.assign(odds=val_df_odds)
        if set(val_df_odds.value_counts().index) == set([1, 0]):
            print(str(clf.get_params()['steps'][1][1]), "doesn't return probabilities, so no streak results will be returned based on this alone.")
            pass
        else:
            print('-----------------------------------')

            print('Val data')
            print('-----------')

            streak_checker(val_df, output=True)
            print('Holdout data')
            print('-----------')

            streak_checker(holdout, output=True)
            
        print('Odds multiplied by PA/G_batter')
        print('-----------')
        
        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        
        print('Odds added to PA/G_batter')
        print('-----------')

        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['L/R_split', 'odds'], combiner='add')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['L/R_split', 'odds'], combiner='add')

In [ ]:
def train_test_split(df, test_size=.25):
    
    '''My simplified train_test_split which just cuts the dataframe by date, in order to accurately evaluate streak results, to roughly the test size requested'''
    
    sample_size = len(X)
    
    train_size = 1 - test_size
    
    train_size = round(train_size * sample_size)
    
    df = df.sort_values(['local_date']).reset_index(drop=True)
    
    date_cutoff = df.iloc[train_size]['local_date']

    train = df[df['local_date'] <= date_cutoff]
    val = df[df['local_date'] > date_cutoff]
    
    X_train = train.drop(['got_a_hit'], axis=1)
    y_train = train['got_a_hit']
    
    X_val = val.drop(['got_a_hit'], axis=1)
    y_val = val['got_a_hit']
    
    return(X_train, X_val, y_train, y_val)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df)

In [ ]:
num_cols = [] # all numeric columns
cols_to_ohe = [] # doubleheader, conditions, stand
cols_to_targ = [] # all other object columns

for c in X_train.columns:
    if X_train[c].dtype in ['float64', 'int64'] and c not in ['game_num', 'batter', 'fielder_2', 'umpire']:
        num_cols.append(c)
    elif len(X_train[c].unique()) < 10:
        cols_to_ohe.append(c)
    else:
        cols_to_targ.append(c)

In [ ]:
num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

ohe_transformer = Pipeline(steps=[
    ('ohe_imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ohencoder', OneHotEncoder(handle_unknown='ignore'))
])

target_encoder = Pipeline(steps=[
    ('freq_enc', ce.target_encoder.TargetEncoder()),
    ('freq_imputer', SimpleImputer(strategy='constant', fill_value=0))
    
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('ohe', ohe_transformer, cols_to_ohe),
        ('target', target_encoder, cols_to_targ)
    ])

# Baseline - dummy models
Pick player with the highest Plate Appearances per Game over the last 2 years

In [ ]:
streak_checker(df, output=True, metric='PA/G_batter')
streak_checker(test_df, output=True, metric='PA/G_batter')

Observations - this seems to trend towards a small group of players except for when a player makes their mLB debut towards the top of the lineup (Wander Franco, Greg Deichmann). Sample size is an issue - could be addressed by either looking at games played or excluding PA/G above 4.9.



Pick player with highest launch speed

In [ ]:
streak_checker(df, output=True, metric='avg_launch_speed')
streak_checker(test_df, output=True, metric='avg_launch_speed')

The metrics point to this being mostly useless which is backed up by the fact that it seems to occasionally choose pitchers (Hunter Strickland, Zack Greinke), when it's not fawning over Aaron Judge. Could be worth excluding pitchers as a class though would be hard to implement with the current dataset, not to mention the fact that 2-way players would need to be excluded from that filter.

In [ ]:
streak_checker(df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')
streak_checker(test_df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')

Exaserbates bias towards small sample sizes, leading part time players to dominate selection rather than proven hitters

Would also be worth checking lefty righty splits

In [ ]:
streak_checker(df, output=True, metric='L/R_split', combiner='add')
streak_checker(test_df, output=True, metric='L/R_split', combiner='add')
streak_checker(df, output=True, metric='L/R_split', combiner='multiply')
streak_checker(test_df, output=True, metric='L/R_split', combiner='multiply')

Nothing stands out in these groups. Seems to be a mix of journeymen and all-stars.

# Model 1 - Decision Tree

In [ ]:
start = datetime.now()
dt = DecisionTreeClassifier()

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', dt)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 2: Logistic Regression

In [ ]:
start = datetime.now()

logreg = LogisticRegression(solver='sag')

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', logreg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 3: SVC too slow, linearSVC doesn't supply probability, so I need to supply a secondary way to select a single batter per day

In [ ]:
start = datetime.now()

Linearsvc = LinearSVC()


clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linearsvc', Linearsvc)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 4: KNN

In [ ]:
start = datetime.now()

knn = KNeighborsClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', knn)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 5: Random Forest

In [ ]:
start = datetime.now()

rfc = RandomForestClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rfc', rfc)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 6: AdaBoost


In [ ]:
start = datetime.now()

ada = AdaBoostClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ada', ada)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 7: GradientBoost


In [ ]:
start = datetime.now()

gbm = GradientBoostingClassifier()
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('gbm', gbm)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 8: XGBoost


In [ ]:
start = datetime.now()

xbg = xgb.XGBClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xbg', xbg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 9: Random Forest with Grid Search


In [ ]:
start = datetime.now()

parameters = {
    'min_samples_split': [3, 5, 100], 
    'n_estimators' : [100, 300],
        'max_depth': [3, 5, 15, 25],

}

run_model(clf, X_train, X_val, y_train, y_val, test_df)

In [ ]:
# to be run daily
def daily_guess():

    # get game data, then weather data, then pitcher data, then batter data

    todays_games = pd.DataFrame()


    for game in statsapi.schedule():
    #     grab 'away_probable_pitcher','home_probable_pitcher','away_name',
    #     'home_name','doubleheader','game_num','venue_name','game_datetime'

    #     df['local_datetime'] = pd.to_datetime(df['local_datetime'])
    #     df['local_date'] = df['local_datetime'].apply(lambda x: x.toordinal())
    #     df['local_datetime'] = df['local_datetime'].apply(lambda x: x.timestamp())

        row = {
            'game_id': game['game_id'],
            'away_probable_pitcher': game['away_probable_pitcher'],
            'home_probable_pitcher': game['home_probable_pitcher'],
            'away_name': game['away_name'],
            'home_name': game['home_name'],
            'doubleheader': game['doubleheader'],
            'game_num': game['game_num'],
            'venue_name': game['venue_name'],
            'game_datetime': game['game_datetime']
        }

        todays_games = todays_games.append(row, ignore_index=True)

    todays_games = todays_games.replace('', np.nan)    

    games_and_weather = time_date_weather_data(todays_games)

    away_pitchers = games_and_weather['away_probable_pitcher']
    home_pitchers = games_and_weather['home_probable_pitcher']
    pitchers_stats = pd.DataFrame()

    for a in away_pitchers:
        print(a)
        if type(a) == float:
            pitchers_stats = pitchers_stats.append(pd.Series(), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats_v2(a, 'pitcher', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **{'probable_pitcher': a}}, ignore_index=True)

    for h in home_pitchers:
        print(h)
        if type(h) == float:
            pitchers_stats = pitchers_stats.append(pd.Series(), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats_v2(h, 'pitcher', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **{'probable_pitcher': h}}, ignore_index=True)

    batting_stats = pd.DataFrame()
    for index, game in games_and_weather.iterrows():
        print(game[['away_name', 'home_name']])
    #     home batters, away pitcher
        batting_team_id = lookup_mlb()[game['home_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            start = datetime.now()
            batting_stats = batting_stats.append({**derived_cumulative_stats_v2(b, 'batter', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **pitchers_stats[pitchers_stats['probable_pitcher'] == game['away_probable_pitcher']], **game[['away_name',
                                             'home_name',
                                             'doubleheader',
                                             'game_num',
                                             'venue_name',
                                             'local_datetime',
                                             'coordinates',
                                             'cloudcover',
                                             'conditions',
                                             'dew',
                                             'heatindex',
                                             'humidity',
                                             'maxt',
                                             'mint',
                                             'precip',
                                             'sealevelpressure',
                                             'snowdepth',
                                             'temp',
                                             'visibility',
                                             'wdir',
                                             'weathertype',
                                             'wgust',
                                             'windchill',
                                             'wspd'
     ]]}, ignore_index=True)
            print(datetime.now()-start)
        #     away batters, home pitcher
        batting_team_id = lookup_mlb()[game['away_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            batting_stats = batting_stats.append({**derived_cumulative_stats_v2(b, 'batter', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **pitchers_stats[pitchers_stats['probable_pitcher'] == game['home_probable_pitcher']], **game[['away_name',
                                             'home_name',
                                             'doubleheader',
                                             'game_num',
                                             'venue_name',
                                             'local_datetime',
                                             'coordinates',
                                             'cloudcover',
                                             'conditions',
                                             'dew',
                                             'heatindex',
                                             'humidity',
                                             'maxt',
                                             'mint',
                                             'precip',
                                             'sealevelpressure',
                                             'snowdepth',
                                             'temp',
                                             'visibility',
                                             'wdir',
                                             'weathertype',
                                             'wgust',
                                             'windchill',
                                             'wspd'
     ]]}, ignore_index=True)

# TODO
# RUN batting_stats through predictors